# Tutorial Julia for Data Science

## Classificazione

Questa parte riguarda come definire e testare modelli di classificazione di dati.  
Vedremo 7 metodi simili tra loro per fare ciò.  
Il problema su cui proveremo la classificazione riguarda il predirre il tipo di un fiore a partire da alcune sue caratteristiche fisiche.  
Iniziamo inizializzando ambiente:  

In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

 Activating environment at `C:\Users\i am\Documents\GitHub\stage-Machine-learning\notebook\tutorial\Project.toml`


In [2]:
using GLMNet
using RDatasets
using MLBase
using Plots
using DecisionTree
using Distances
using NearestNeighbors
using Random
using LinearAlgebra
using DataStructures
using LIBSVM

Carichiamo i dati:

In [7]:
iris = dataset("datasets", "iris")
X = Matrix(iris[:,1:4])#dati in matrice
irislabels = iris[:,5];#vogliamo predirre colonna 5: species
irislabelsmap = labelmap(irislabels)#mappiamo le classi in numeri
y = labelencode(irislabelsmap, irislabels);#convertiamo colonna 5 in numeri

Ci serve un dataset per modellare i classificatori e un dataset per testarli quindi prendiamo i dati che abbiamo e dividiamoli.  
La funzione che useremo, randsubseq(iterable, percentage) , scorre i dati di un iterable in input e, uno ad uno, li copia in un iterable di output solo con probabilità data in input.  

In [9]:
function perclass_splits(y,at)
    uids = unique(y)
    keepids = []
    for ui in uids
        curids = findall(y.==ui)
        rowids = randsubseq(curids, at) # randsubset prende una struct, scorre gli elem e, ogni elem è ritornato con p probabilità

        push!(keepids,rowids...)
    end
    return keepids
end

trainids = perclass_splits(y,0.7)
testids = setdiff(1:length(y),trainids);

Ci serve una funzione che ci aiuti a giudicare quanto un modello possa essere buono.  
La funzione che useremo calcola semplicemente il rapporto $ \frac{\text{predizioni corrette}}{\text{predizioni totali}}$ 

In [11]:
findaccuracy(predictedvals,groundtruthvals) = sum(predictedvals.==groundtruthvals)/length(groundtruthvals)

findaccuracy (generic function with 1 method)

Infine, i numeri che i nostri modelli predirranno non saranno probabilmente interi, quindi creiamo una piccola funzione che converta numeri continui in discreti:

In [12]:
assign_class(predictedvalue) = argmin(abs.(predictedvalue .- [1,2,3]))

assign_class (generic function with 1 method)

### 1- Lasso


In [13]:
path = glmnet(X[trainids,:], y[trainids])#usiamo trainids per creare modello
cv = glmnetcv(X[trainids,:], y[trainids])#trova lambda migliore

#glmnet ritorna più landa per diversi gradi di accuratezza, selezioniamo il migliore
mylambda = path.lambda[argmin(cv.meanloss)]
path = glmnet(X[trainids,:], y[trainids],lambda=[mylambda])

Least Squares GLMNet Solution Path (1 solutions for 4 predictors in 78 passes):
─────────────────────────────
     df   pct_dev           λ
─────────────────────────────
[1]   3  0.916014  0.00702171
─────────────────────────────

In [14]:
q = X[testids,:];
predictions_lasso = GLMNet.predict(path,q)#applichiamo modello su testids

predictions_lasso = assign_class.(predictions_lasso)#modello restituisce numeri continui, portiamoli alle etichette
findaccuracy(predictions_lasso,y[testids])

0.9777777777777777

### 2- Ridge
Uguale a Lasso ma con un parametro in più: alpha=0 a glmnet:

In [15]:
path = glmnet(X[trainids,:], y[trainids],alpha=0);
cv = glmnetcv(X[trainids,:], y[trainids],alpha=0)
mylambda = path.lambda[argmin(cv.meanloss)]
path = glmnet(X[trainids,:], y[trainids],alpha=0,lambda=[mylambda]);
q = X[testids,:];
predictions_ridge = GLMNet.predict(path,q)
predictions_ridge = assign_class.(predictions_ridge)
findaccuracy(predictions_ridge,y[testids])

0.9555555555555556

### 3- Rete Elastica
Simile, ora alpha sarà 0.5.

In [16]:
path = glmnet(X[trainids,:], y[trainids],alpha=0.5);
cv = glmnetcv(X[trainids,:], y[trainids],alpha=0.5)
mylambda = path.lambda[argmin(cv.meanloss)]
path = glmnet(X[trainids,:], y[trainids],alpha=0.5,lambda=[mylambda]);
q = X[testids,:];
predictions_EN = GLMNet.predict(path,q)
predictions_EN = assign_class.(predictions_EN)
findaccuracy(predictions_EN,y[testids])

0.9555555555555556

### 4- Decision Tree
Questa volta usiamo il pacchetto DecisionTree per creare un albero decisionale.

In [18]:
model = DecisionTreeClassifier(max_depth=2)
DecisionTree.fit!(model, X[trainids,:], y[trainids])
q = X[testids,:];
predictions_DT = DecisionTree.predict(model, q)
findaccuracy(predictions_DT,y[testids])

0.9555555555555556

### 5- Foreste casuali
Sempre dallo stesso pacchetto, usiamo il metodo per le random forest.

In [19]:
model = RandomForestClassifier(n_trees=20)#dal pkg decisiontree
DecisionTree.fit!(model, X[trainids,:], y[trainids])
q = X[testids,:];
predictions_RF = DecisionTree.predict(model, q)
findaccuracy(predictions_RF,y[testids])

0.9555555555555556

### 6- Nearest neighbohor
Vediamo, dal pacchettoi NearestNeighbors

In [20]:
Xtrain = X[trainids,:]
ytrain = y[trainids]
kdtree = KDTree(Xtrain')
queries = X[testids,:]
idxs, dists = knn(kdtree, queries', 5, true)
c = ytrain[hcat(idxs...)]
possible_labels = map(i->counter(c[:,i]),1:size(c,2))
predictions_NN = map(i->parse(Int,string(argmax(DataFrame(possible_labels[i])[1,:]))),1:size(c,2))
findaccuracy(predictions_NN,y[testids])

0.9777777777777777

### 7- Support Vector Machines
Quest'ultimo metodo è dal pacchetto LIBSVM

In [21]:
Xtrain = X[trainids,:]
ytrain = y[trainids]
model = svmtrain(Xtrain', ytrain)
predictions_SVM, decision_values = svmpredict(model, X[testids,:]')
findaccuracy(predictions_SVM,y[testids])

0.9555555555555556

### Conclusione
Vediamo ora come tutti i metodi si comparano tra loro:

In [23]:
overall_accuracies = zeros(7)
methods = ["lasso","ridge","EN", "DT", "RF","kNN", "SVM"]
ytest = y[testids]
overall_accuracies[1] = findaccuracy(predictions_lasso,ytest)
overall_accuracies[2] = findaccuracy(predictions_ridge,ytest)
overall_accuracies[3] = findaccuracy(predictions_EN,ytest)
overall_accuracies[4] = findaccuracy(predictions_DT,ytest)
overall_accuracies[5] = findaccuracy(predictions_RF,ytest)
overall_accuracies[6] = findaccuracy(predictions_NN,ytest)
overall_accuracies[7] = findaccuracy(predictions_SVM,ytest)
hcat(methods, overall_accuracies)

7×2 Array{Any,2}:
 "lasso"  0.977778
 "ridge"  0.955556
 "EN"     0.955556
 "DT"     0.955556
 "RF"     0.955556
 "kNN"    0.977778
 "SVM"    0.955556

Nearest Neighbohor e Lasso sembrerebbero essere i migliori per questi dati.